# Function Example

Functions are nice tools both for creating generalizable code, and also just for organizing your work. Because a function is a bundle of code that does one thing, putting parts of your work in functions can be helpful in defining the goal of that chunk of code. 

To illustrate one use of a function, we'll write a function that reads and manipulates a .csv file. We can then put this in a for loop to iterate over several files with a similar structure and combine the resulting data frames into one data frame.

As you'll see, one *could* cram all the code we're gonna write directly into the for-loop at the end, but by breaking out part of it into a function, the problem is more easily broken into smaller parts. 

What follows is an example of how you might use functions in a real data science workflow. The goal is not for you to be comfortable with everything that you see -- for example, the code uses a couple of tricks for manipulating data on dates -- but rather just to get to see what some of this stuff looks like!

### Reading several files


Begin by downloading a [.zip file with service request data from NYC](https://github.com/nickeubank/computational_methods_boot_camp/blob/main/source/data/nyc-311-sample.zip). The zip file contains six files for years 2004-2009, each with 10,000 observations. The data are originally from [NYC's Open Data portal](https://nycopendata.socrata.com/data?cat=social%20services), which hosts datasets with millions of service requests filed by residents through the city's 311 program. For the purpose of this example, I have taken a random sample of 10,000 for each year.

Here's what the 2004 file looks like (the other years have the same structure).

In [1]:
url2004 <- "https://raw.githubusercontent.com/nickeubank/computational_methods_boot_camp/main/source/data/nyc-311-sample/nyc-311-2004-sample.csv"
nyc04 <- read.csv(url2004)
head(nyc04)

,Unique.Key,Created.Date,Closed.Date,Complaint.Type,Location
,<int>,<chr>,<chr>,<chr>,<chr>
1,4735434,01/23/2004 12:00 AM,02/02/2004 12:00 AM,Boilers,"(40.71511134258793, -73.98998982667266)"
2,7547062,06/04/2004 12:00 AM,06/09/2004 12:00 AM,HEATING,"(40.871781348425515, -73.88238262118011)"
3,5050661,08/04/2004 12:00 AM,08/06/2004 12:00 AM,General Construction/Plumbing,"(40.59418801428136, -73.80082145383885)"
4,7281795,11/26/2004 12:00 AM,12/10/2004 12:00 AM,PLUMBING,"(40.85911979460089, -73.90605127158484)"
5,1443894,08/22/2004 12:00 AM,08/22/2004 12:00 AM,Noise - Street/Sidewalk,"(40.54800892371052, -74.17041676351323)"
6,3244577,12/02/2004 12:00 AM,12/15/2004 12:00 AM,Noise,


The variables in the data are as follows: 

* `Unique.Key`: An id number unique to each request.
* `Created.Date`: The date the request was filed in the 311 system.
* `Closed.Date`: The date the request was resolved by city workers (`NA`
implies that it was never resolved).
* `Complaint.Type`: The subject of the complaint.
* `Location`: Coordinates that give the location of the service issue.

Our goal with the function is to read the file and clean it. In particular,
we want to convert the `Created.Date` and `Closed.Date` variables so that
R recognizes them as dates. From these variables, we can then calculate
measures of *government responsiveness*: (1) how many days it took city
workers to resolve a request, and (2) whether or not a request was resolved
within a week. 

In [2]:
library(lubridate) # to work with dates


Attaching package: ‘lubridate’




The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [3]:
# Load required packages

# Create a function that reads and cleans a service request file.
# The input is the name of a service request file and the
# output is a data frame with cleaned variables.
clean_dta <- function(file_name) {

    # Read the file and save it to an object called 'dta'
    dta <- read.csv(file_name)

    # Clean the dates in the dta file and generate responsiveness measures
    # mdy(substring(dta$Created.Date, 1, 10)) pulls just the month-day-year
    # from our columns with dates, then `mdy` tells R to read it as a date
    # in month-day-year format.

    dta$opened <- mdy(substring(dta$Created.Date, 1, 10))
    dta$closed <- mdy(substring(dta$Closed.Date, 1, 10))

    # Number of days between an issue opens and is resolved.
    dta$resptime <- as.numeric(difftime(dta$closed, dta$opened, units = "days"))

    # Create indicator of whether solved within 7 days.
    # responses in less than 0 is bad data.
    dta[dta$resptime < 0 | is.na(dta$resptime), "resptime"] <- NA
    dta$solvedin7 <- as.numeric(dta$resptime <= 7)

    # Return the cleaned data
    return(dta)
}

Let's test the function on the 2004 data:

In [4]:
# Execute function on the 2004 data
nyc04 <- clean_dta(url2004)
head(nyc04)

,Unique.Key,Created.Date,Closed.Date,Complaint.Type,Location,opened,closed,resptime,solvedin7
,<int>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>,<dbl>
1,4735434,01/23/2004 12:00 AM,02/02/2004 12:00 AM,Boilers,"(40.71511134258793, -73.98998982667266)",2004-01-23,2004-02-02,10,0
2,7547062,06/04/2004 12:00 AM,06/09/2004 12:00 AM,HEATING,"(40.871781348425515, -73.88238262118011)",2004-06-04,2004-06-09,5,1
3,5050661,08/04/2004 12:00 AM,08/06/2004 12:00 AM,General Construction/Plumbing,"(40.59418801428136, -73.80082145383885)",2004-08-04,2004-08-06,2,1
4,7281795,11/26/2004 12:00 AM,12/10/2004 12:00 AM,PLUMBING,"(40.85911979460089, -73.90605127158484)",2004-11-26,2004-12-10,14,0
5,1443894,08/22/2004 12:00 AM,08/22/2004 12:00 AM,Noise - Street/Sidewalk,"(40.54800892371052, -74.17041676351323)",2004-08-22,2004-08-22,0,1
6,3244577,12/02/2004 12:00 AM,12/15/2004 12:00 AM,Noise,,2004-12-02,2004-12-15,13,0


The cleaned dataset has four new variables:

* `opened`: The date the request was filed in date format. 
* `closed`: The date the request was resolved in date format. 
* `resptime`: The number of days it took to resolve the request (`closed` - `opened`).
* `solvedin7`: A dummy variable equal to 1 if the request was solved within a week
  and 0 otherwise. 

We can now use this function on all the six files using a for loop, or something called lapply() (Read more about `lapply()`
[here](http://www.r-bloggers.com/using-apply-sapply-lapply-in-r/)).

In [5]:
# loop over and collect in a list!

url_stem <- "https://raw.githubusercontent.com/nickeubank/computational_methods_boot_camp/main/source/data/nyc-311-sample/nyc-311-"
url_suffix <- "-sample.csv"

# Get first one so we can append others to the bottom
nyc_all <- clean_dta(paste0(url_stem, 2004, url_suffix))

for (year in 2005:2009) {
    url <- paste0(url_stem, year, url_suffix)
    new_data <- clean_dta(url)
    nyc_all <- rbind(nyc_all, new_data)
}

In [6]:
# 10 random rows
nyc_all[sample(nrow(nyc_all), 10), ]

,Unique.Key,Created.Date,Closed.Date,Complaint.Type,Location,opened,closed,resptime,solvedin7
,<int>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<dbl>,<dbl>
51771,14194945,06/16/2009 12:00:00 AM,06/16/2009 12:00:00 AM,Traffic Signal Condition,,2009-06-16,2009-06-16,0,1
18325,8716616,12/05/2005 12:00 AM,12/07/2005 12:00 AM,HEATING,"(40.856497346130325, -73.8920651569873)",2005-12-05,2005-12-07,2,1
25442,8099179,07/20/2006 12:00 AM,07/26/2006 12:00 AM,GENERAL CONSTRUCTION,"(40.76039373041877, -73.96141567984733)",2006-07-20,2006-07-26,6,1
30947,9108747,08/15/2007 12:00:00 AM,08/15/2007 12:00:00 AM,Homeless Encampment,"(40.74152553460752, -73.97978045495324)",2007-08-15,2007-08-15,0,1
58136,14834317,09/10/2009 12:00:00 AM,09/10/2009 12:00:00 AM,Street Light Condition,"(40.76350927910743, -73.7719160058321)",2009-09-10,2009-09-10,0,1
31439,9534693,10/07/2007 12:00:00 AM,10/08/2007 12:00:00 AM,Blocked Driveway,"(40.730748472480975, -73.95594850807463)",2007-10-07,2007-10-08,1,1
59432,13536197,04/09/2009 12:00:00 AM,04/10/2009 12:00:00 AM,Street Light Condition,"(40.54262605600831, -74.20112967417194)",2009-04-09,2009-04-10,1,1
1898,8366246,08/13/2004 12:00 AM,10/18/2004 12:00 AM,NONCONST,"(40.68460998586983, -73.95159094788242)",2004-08-13,2004-10-18,66,0
44365,10749324,03/19/2008 12:00:00 AM,07/13/2011 12:00:00 AM,Consumer Complaint,"(40.6340115466154, -74.02676583938911)",2008-03-19,2011-07-13,1211,0


Ta-da! We cleaned a whole bunch of different datasets and merged them all together in only handful of lines of code!

</div>


